In [ ]:
using Pkg
Pkg.add("UnROOT")

In [ ]:
using UnROOT

In [ ]:
file = ROOTFile("data/SMHiggsToZZTo4L.root")

In [ ]:
events = LazyTree(file, "Events")

In [ ]:
using AwkwardArray

In [ ]:
muons = AwkwardArray.RecordArray(
        NamedTuple{(:pt, :eta, :phi, :mass, :charge, :isolation)}((
            AwkwardArray.from_iter(events.Muon_pt),
            AwkwardArray.from_iter(events.Muon_eta), 
            AwkwardArray.from_iter(events.Muon_phi), 
            AwkwardArray.from_iter(events.Muon_mass), 
            AwkwardArray.from_iter(events.Muon_charge), 
            AwkwardArray.from_iter(events.Muon_pfRelIso03_all),
        )
    ))

In [ ]:
using PythonCall

In [ ]:
four_muons = AwkwardArray.ListArray{
    AwkwardArray.Index64,
    AwkwardArray.TupleArray{Tuple{
        AwkwardArray.PrimitiveArray{Int64},
        AwkwardArray.PrimitiveArray{Int64},
        AwkwardArray.PrimitiveArray{Int64},
        AwkwardArray.PrimitiveArray{Int64}}
    }
}()

In [ ]:
push!(AwkwardArray.slot(four_muons.content, 1), 1)
push!(AwkwardArray.slot(four_muons.content, 2), 2)
push!(AwkwardArray.slot(four_muons.content, 3), 3)
push!(AwkwardArray.slot(four_muons.content, 4), 4)

In [ ]:
four_muons = AwkwardArray.ListArray{
    AwkwardArray.Index64,
    AwkwardArray.PrimitiveArray{Int64},
}()

In [ ]:
function find_4lep(events_leptons)

    for leptons in events_leptons
        nlep = length(leptons[:charge])
        for i0 in 1:nlep
            for i1 in (i0 + 1):nlep
                if leptons[:charge][i0] + leptons[:charge][i1] != 0
                    continue
                end
                for i2 in 1:nlep
                    for i3 in (i2 + 1):nlep
                        if length(Set([i0, i1, i2, i3])) < 4
                            continue
                        end
                        if leptons[:charge][i2] + leptons[:charge][i3] != 0
                            continue
                        end
                        # push!(AwkwardArray.slot(four_muons.content, 1), (i0 - 1))  # Julia is 1-based, subtract 1 for 0-based indexing
                        # push!(AwkwardArray.slot(four_muons.content, 2), (i1 - 1))
                        # push!(AwkwardArray.slot(four_muons.content, 3), (i2 - 1))
                        # push!(AwkwardArray.slot(four_muons.content, 4), (i3 - 1))
                        
                        # AwkwardArray.end_tuple!(four_muons.content)
                        push!(four_muons.content, (i0 - 1)) # Julia is 1-based, subtract 1 for 0-based indexing
                        push!(four_muons.content, (i1 - 1))
                        push!(four_muons.content, (i2 - 1))
                        push!(four_muons.content, (i3 - 1))
                        AwkwardArray.end_list!(four_muons)
                    end
                end
            end 
        end
    end
end

In [ ]:
find_4lep(muons[1:10])

In [ ]:
four_muons